### Original code from class

In [1]:
"""
Knowledge-Based Matchmaking System: The system will match user profiles based on weighted 
compatibility scores across multiple attributes.
The KBS maintains profiles and user-specific rules to determine matches.
"""

class KnowledgeBase:
    """
    Stores and manages user profiles with unique identifiers.
    Acts as the knowledge repository for the matchmaking system.
    """

    def __init__(self):
        self.profiles = {}  # Dictionary to store profiles (profile_id: profile_data)
        self.next_id = 1  # Auto-incrementing ID counter

    def add_profile(self, profile_data):
        """
        Adds a new profile to the knowledge base with automatic ID assignment.
        :param profile_data: Dictionary containing profile attributes.
        :return: Assigned profile ID.
        """
        profile_id = self.next_id
        self.profiles[profile_id] = {'id': profile_id, **profile_data}  # Add profile data with ID
        self.next_id += 1
        return profile_id

    def get_profile(self, profile_id):
        """
        Retrieves a profile by ID.
        """
        return self.profiles.get(profile_id)

    def get_all_profiles(self):
        """
        Retrieves all profiles for matchmaking.
        """
        return list(self.profiles.values())


class Matchmaker:
    """
    Implements matchmaking logic using compatibility rules and weights.
    Leverages the knowledge base to find optimal matches for a given profile.
    """

    def __init__(self, knowledge_base, weights=None, max_age_diff=20):
        """
        :param knowledge_base: KnowledgeBase instance.
        :param weights: Dictionary containing attribute weights for matchmaking (sum to 1).
        :param max_age_diff: Maximum age difference allowed between profiles.
        """
        self.knowledge_base = knowledge_base
        self.weights = weights or {'age': 0.3, 'interests': 0.5, 'location': 0.2}  # Default weights
        self.max_age_diff = max_age_diff  # Maximum age difference allowed

    def calculate_compatibility(self, profile_a, profile_b):
        """
        Computes weighted compatibility scores between two profiles.
        Implements domain-specific matching rules:
        - Age: Normalized inverse age gap.
        - Interests: Jaccard similarity of interests.
        - Location: Exact match check.
        """

        # Age compatibility (Normalized inverse difference)
        age_diff = abs(profile_a['age'] - profile_b['age'])  # Absolute age difference
        age_score = max(0, 1 - (age_diff / self.max_age_diff)) * self.weights['age']  # Normalized inverse difference

        # Interest compatibility (Jaccard similarity)
        interests_a = set(profile_a['interests'])
        interests_b = set(profile_b['interests'])
        intersection = len(interests_a & interests_b)  # Common interests
        union = len(interests_a | interests_b)  # All interests
        jaccard_score = (intersection / union) * self.weights['interests'] if union > 0 else 0  # Jaccard similarity

        # Location compatibility
        location_score = self.weights['location'] if profile_a['location'] == profile_b['location'] else 0

        return age_score + jaccard_score + location_score

    def find_matches(self, user_id, top_n=5):
        """
        Finds the top N matches for a given user based on compatibility scores.
        :param user_id: Profile ID of the user to find matches for.
        :param top_n: Number of top matches to return.
        :return: List of (profile_id, compatibility_score) tuples.
        """
        target_profile = self.knowledge_base.get_profile(user_id)
        if not target_profile:
            return []

        matches = []
        for profile in self.knowledge_base.get_all_profiles():
            if profile['id'] == user_id:
                continue  # Skip the same profile

            score = self.calculate_compatibility(target_profile, profile)
            matches.append((profile['id'], round(score, 2)))

        # Sort matches by score and return top N
        matches.sort(key=lambda x: x[1], reverse=True)
        return matches[:top_n]


if __name__ == "__main__":
    # Initialize the Knowledge Base
    kb = KnowledgeBase()

    # Populate with sample profiles
    profiles = [
        {'age': 25, 'interests': ['music', 'sports'], 'location': 'New York'},
        {'age': 28, 'interests': ['music', 'travel'], 'location': 'Los Angeles'},
        {'age': 35, 'interests': ['sports', 'travel'], 'location': 'New York'},
        {'age': 40, 'interests': ['music', 'sports'], 'location': 'Chicago'},
    ]

    for p in profiles:
        kb.add_profile(p)

    # Initialize the Matchmaker
    matchmaker = Matchmaker(
        knowledge_base=kb,
        weights={'age': 0.25, 'interests': 0.6, 'location': 0.15},
        max_age_diff=25
    )

    # Find matches for the first profile
    print("Top Matches for Profile 1:")
    for match in matchmaker.find_matches(1, top_n=3):
        print(f"Profile {match[0]} with Compatibility Score: {match[1]}")


Top Matches for Profile 1:
Profile 4 with Compatibility Score: 0.7
Profile 3 with Compatibility Score: 0.5
Profile 2 with Compatibility Score: 0.42


### Updated code to turn it into a dating matchmaker KBS

In [ ]:
import random

class KnowledgeBase:
    """
    Stores and manages user profiles with unique identifiers.
    Acts as the knowledge repository for the matchmaking system.
    """

    def __init__(self):
        self.profiles = {}  # Dictionary to store profiles (profile_id: profile_data)
        self.next_id = 1  # Auto-incrementing ID counter

    def add_profile(self, profile_data):
        """
        Adds a new profile to the knowledge base with automatic ID assignment.
        :param profile_data: Dictionary containing profile attributes.
        :return: Assigned profile ID.
        """
        profile_id = self.next_id
        self.profiles[profile_id] = {'id': profile_id, **profile_data}
        self.next_id += 1
        return profile_id

    def get_profile(self, profile_id):
        """ Retrieves a profile by ID. """
        return self.profiles.get(profile_id)

    def get_all_profiles(self):
        """ Retrieves all profiles for matchmaking. """
        return list(self.profiles.values())


class Matchmaker:
    """
    Implements Tinder-like matchmaking using compatibility rules and weighted attributes.
    """

    def __init__(self, knowledge_base, weights=None, max_age_diff=10):
        """
        :param knowledge_base: KnowledgeBase instance.
        :param weights: Dictionary containing attribute weights for matchmaking (sum to 1).
        :param max_age_diff: Maximum age difference allowed between profiles.
        """
        self.knowledge_base = knowledge_base
        self.weights = weights or {
            'age': 0.2, 'interests': 0.3, 'location': 0.15,
            'education': 0.1, 'occupation': 0.1, 'lifestyle': 0.15
        }
        self.max_age_diff = max_age_diff

    def calculate_compatibility(self, profile_a, profile_b):
        """
        Computes weighted compatibility scores between two profiles.
        """

        # Check Gender Preference
        if profile_a['preferred_gender'] not in [profile_b['gender'], "Any"]:
            return 0  # No match if gender preference doesn't align

        if profile_b['preferred_gender'] not in [profile_a['gender'], "Any"]:
            return 0

        # Age Compatibility (Normalized inverse difference)
        age_diff = abs(profile_a['age'] - profile_b['age'])
        age_score = max(0, 1 - (age_diff / self.max_age_diff)) * self.weights['age']

        # Interest Compatibility (Jaccard similarity)
        interests_a = set(profile_a['interests'])
        interests_b = set(profile_b['interests'])
        intersection = len(interests_a & interests_b)
        union = len(interests_a | interests_b)
        jaccard_score = (intersection / union) * self.weights['interests'] if union > 0 else 0

        # Location Compatibility
        location_score = self.weights['location'] if profile_a['location'] == profile_b['location'] else 0

        # Education Compatibility
        education_score = self.weights['education'] if profile_a['education'] == profile_b['education'] else 0

        # Occupation Compatibility
        occupation_score = self.weights['occupation'] if profile_a['occupation'] == profile_b['occupation'] else 0

        # Lifestyle Compatibility (Exact Match)
        lifestyle_score = self.weights['lifestyle'] if profile_a['lifestyle'] == profile_b['lifestyle'] else 0

        # Final Weighted Score
        return age_score + jaccard_score + location_score + education_score + occupation_score + lifestyle_score

    def find_matches(self, user_id, top_n=5):
        """
        Finds the top N matches for a given user based on compatibility scores.
        """
        target_profile = self.knowledge_base.get_profile(user_id)
        if not target_profile:
            return []

        matches = []
        for profile in self.knowledge_base.get_all_profiles():
            if profile['id'] == user_id:
                continue  # Skip self-matching

            score = self.calculate_compatibility(target_profile, profile)
            if score > 0:  # Only include valid matches
                matches.append((profile['id'], round(score, 2)))

        # Sort matches by score and return top N
        matches.sort(key=lambda x: x[1], reverse=True)
        return matches[:top_n]


if __name__ == "__main__":
    # Initialize the Knowledge Base
    kb = KnowledgeBase()

    # Populate with sample profiles
    sample_profiles = [
        {'age': 25, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'New York',
         'interests': ['music', 'sports'], 'education': 'Bachelor', 'occupation': 'Engineer',
         'relationship_goals': 'Long-term', 'lifestyle': 'Active'},
        
        {'age': 28, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Los Angeles',
         'interests': ['music', 'travel'], 'education': 'Master', 'occupation': 'Designer',
         'relationship_goals': 'Casual', 'lifestyle': 'Active'},

        {'age': 32, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'New York',
         'interests': ['sports', 'travel'], 'education': 'PhD', 'occupation': 'Professor',
         'relationship_goals': 'Marriage', 'lifestyle': 'Non-Smoker'},

        {'age': 27, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Chicago',
         'interests': ['music', 'sports'], 'education': 'Bachelor', 'occupation': 'Teacher',
         'relationship_goals': 'Long-term', 'lifestyle': 'Smoker'},
    ]

    for p in sample_profiles:
        kb.add_profile(p)

    # Initialize the Matchmaker
    matchmaker = Matchmaker(
        knowledge_base=kb,
        weights={'age': 0.2, 'interests': 0.3, 'location': 0.15, 'education': 0.1, 'occupation': 0.1, 'lifestyle': 0.15},
        max_age_diff=10
    )

    # Find matches for the first profile
    print("Top Matches for Profile 1:")
    for match in matchmaker.find_matches(1, top_n=3):
        print(f"Profile {match[0]} with Compatibility Score: {match[1]}")
